In [1]:
import pandas as pd
import mxnet
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

from sklearn.model_selection import train_test_split


device = torch.device("cuda:1")
print(f"Using {device}")

2022-07-01 19:32:00.198580: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


Using cuda:1


In [2]:
data = pd.read_pickle("../result/tokenized_test_data.pkl")

In [3]:
#앞쪽 5만은 학습사용 뒤쪽 5만은 학습 사용하지 않음
pred_seen = data[:50000]
pred_unseen = data[50000:]

#어떤것으로 테스트할까요?
pred_data = data[['token', '접수기관']]
#pred_data = pred_seen[['token', '접수기관']]
#pred_data = pred_unseen[['token', '접수기관']]

pred_data.reset_index(inplace = True, drop = True)
pred_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   token   100000 non-null  object
 1   접수기관    100000 non-null  object
dtypes: object(2)
memory usage: 1.5+ MB


In [4]:
label_to_int = pd.read_pickle("../result/label_dict.pkl")

In [5]:
label_to_int

{'식품의약품안전처': 0,
 '국토교통부': 1,
 '한국토지주택공사': 2,
 '경상남도 양산시': 3,
 '경상남도 창원시': 4,
 '인사혁신처': 5,
 '경기도 성남시': 6,
 '부산광역시 부산진구': 7,
 '경찰청': 8,
 '고용노동부': 9,
 '환경부': 10,
 '금융위원회': 11,
 '대한법률구조공단': 12,
 '경기도 시흥시': 13,
 '경기도 용인시': 14,
 '경기도 안산시': 15,
 '행정안전부': 16,
 '인천광역시': 17,
 '경상북도 경산시': 18,
 '대전광역시 서구': 19,
 '한국공정거래조정원': 20,
 '전라남도 보성군': 21,
 '경기도 화성시': 22,
 '전라북도 익산시': 23,
 '경기도 의정부시': 24,
 '부산광역시 사하구': 25,
 '경기도 수원시': 26,
 '국세청': 27,
 '국민권익위원회': 28,
 '울산광역시 남구': 29,
 '경기도 평택시': 30,
 '충청남도 계룡시': 31,
 '과학기술정보통신부': 32,
 '서울특별시 강남구': 33,
 '경상북도 구미시': 34,
 '서울특별시': 35,
 '병무청': 36,
 '세종특별자치시': 37,
 '경기도 안양시': 38,
 '제주특별자치도 제주시': 39,
 '경기주택도시공사': 40,
 '전라남도 여수시': 41,
 '교육부': 42,
 '외교부': 43,
 '해양수산부': 44,
 '관세청': 45,
 '보건복지부': 46,
 '경기도교육청': 47,
 '전라남도 무안군': 48,
 '국방부': 49,
 '법원행정처(대법원)': 50,
 '경기도 김포시': 51,
 '경기도 광주시': 52,
 '서울특별시교육청': 53,
 '금융감독원': 54,
 '한국교통안전공단': 55,
 '한국환경공단': 56,
 '산업통상자원부': 57,
 '서울특별시 관악구': 58,
 '경기도 파주시': 59,
 '강원도 강릉시': 60,
 '한국소비자원': 61,
 '대검찰청': 62,
 '경기도 고양시': 63,
 '경기도 

In [6]:
idx_list = []
for idx, item in enumerate(pred_data['접수기관']):
    if item in label_to_int.keys():
        idx_list.append(idx)

In [7]:
print(len(pred_data) , len(idx_list))

100000 100000


In [8]:
pred_data = pred_data.iloc[idx_list, :]

In [9]:
pred_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   token   100000 non-null  object
 1   접수기관    100000 non-null  object
dtypes: object(2)
memory usage: 2.3+ MB


In [10]:
def department_to_int(x):
    try:
        return label_to_int[x]
    except:
        print('error')

In [11]:
pred_data.head(3)

,token,접수기관
0,"[모금, 정밀, 안전, 진단, 비용, 억, 원, 배상, 정부, 국민, 정책, 신뢰,...",국토교통부
1,"[재건축, 안전, 진단, 적용, 철회, 안전, 진단, 강화, 규정, 안전, 진단, ...",국토교통부
2,"[재건축, 안전, 진단, 적용, 철회, 안전, 진단, 강화, 규정, 안전, 진단, ...",국토교통부


In [12]:
pred_data['접수기관'] = pred_data['접수기관'].apply(lambda x : department_to_int(x))
pred_data

,token,접수기관
0,"[모금, 정밀, 안전, 진단, 비용, 억, 원, 배상, 정부, 국민, 정책, 신뢰,...",1
1,"[재건축, 안전, 진단, 적용, 철회, 안전, 진단, 강화, 규정, 안전, 진단, ...",1
2,"[재건축, 안전, 진단, 적용, 철회, 안전, 진단, 강화, 규정, 안전, 진단, ...",1
3,"[남양주, 왕숙, 도시, 성공, 여부, 교통, 대책, 효율, 교통, 체증, 분산, ...",2
4,"[장지, 입체, 사업, 관련, 협의, 체, 구성, 사업, 피해, 인접, 단지, 대표...",35
...,...,...
99995,"[민원, 유입, 경로, 모바일, 사고, 발생, 지역, 서울특별시, 구로구, 구로동,...",177
99996,"[민원, 유입, 경로, 모바일, 사고, 발생, 지역, 경상남도, 양산시, 중부동, ...",3
99997,"[블럭, 주, 입구, 하나, 추가, 하나, 수, 블럭, 교통, 혼잡, 야기, 것, 문]",2
99998,"[민원, 유입, 경로, 모바일, 사고, 발생, 지역, 경기도, 화성시, 신동, 동탄...",22


In [13]:
class BERTDataset(Dataset):
    def __init__(self, dataset,bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([" ".join(dataset.iloc[i]['token'])]) for i in range(len(dataset))]
        self.labels = [np.int32(dataset.iloc[i]['접수기관']) for i in range(len(dataset))]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

print('get bertmodel and vocab')
bertmodel, vocab = get_pytorch_kobert_model()


get bertmodel and vocab
using cached model. /home/mglee/VSCODE/git_folder/complain_department_classification/code/.cache/kobert_v1.zip
using cached model. /home/mglee/VSCODE/git_folder/complain_department_classification/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [14]:
max_grad_norm = 1
log_interval = 1000
warmup_ratio = 0.1
batch_size = 16
max_len = 512

print("data setting")
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

pred_data_B = BERTDataset(pred_data, tok, max_len, True, False)

pred_dataloader = torch.utils.data.DataLoader(
    pred_data_B, batch_size = batch_size, num_workers = 8)


data setting
using cached model. /home/mglee/VSCODE/git_folder/complain_department_classification/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [16]:
for i in pred_dataloader:
    print(i[0]) # input
    print(i[1]) # ?
    print(i[2]) # label
    print(i[3]) # ?
    break

tensor([[   2, 3417, 7096,  ...,    1,    1,    1],
        [   2, 2173, 3589,  ...,    1,    1,    1],
        [   2, 3135, 5724,  ...,    1,    1,    1],
        ...,
        [   2,  517, 5468,  ...,    1,    1,    1],
        [   2, 5112, 7724,  ...,    1,    1,    1],
        [   2,  517, 5712,  ...,    1,    1,    1]], dtype=torch.int32)
tensor([  6,  92, 125, 129,  35, 102,  32, 298,  37,  51, 512, 246, 512, 214,
        183, 103], dtype=torch.int32)
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32)
tensor([  1,  39,   1, 170, 105, 229,   2,  52,  63,  69,  80,  33,  26,   1,
         10, 214], dtype=torch.int32)


In [15]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = len(label_to_int),   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


In [16]:
PATH = '../result/kobert/'

pred_model = torch.load(PATH + 'KoBERT_0629_e10_py.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
# pred_model.load_state_dict(torch.load(PATH + 'Kobert_0627_py_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장

# checkpoint = torch.load(PATH + 'all.tar')   # dict 불러오기
# pred_model.load_state_dict(checkpoint['model'])
#optimizer.load_state_dict(checkpoint['optimizer'])

In [17]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    #print(f"acc {train_acc} /  mi {max_indices} ==  {Y}")
    return train_acc

### Data Set만 준비한다면 여기를 계속 다른 인풋으로 돌려보면서 확인 가능

In [18]:
out_lst = []
pred_acc = 0
pred_model.eval()
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(pred_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = pred_model(token_ids, valid_length, segment_ids)
    out_lst.append(out.data.cpu())
    pred_acc += calc_accuracy(out, label)
    max_vals, max_indices = torch.max(out, 1)
final_acc = pred_acc / (batch_id+1)
print(f"accuracy of test data is {final_acc}")

  0%|          | 0/6250 [00:00<?, ?it/s]

accuracy of test data is 0.87423


In [19]:
int_to_label = {v:k for k,v in label_to_int.items()}

In [20]:
pred = []
for batch in out_lst:
    for item in batch:
        pred.append(int_to_label[int(np.argmax(item))])

In [21]:
pred_data['pred'] = pred

In [22]:
actual = []

for i in pred_data['접수기관']:
    actual.append(int_to_label[i])

In [23]:
pred_data['real'] = actual

In [24]:
pred_data

,token,접수기관,pred,real
0,"[모금, 정밀, 안전, 진단, 비용, 억, 원, 배상, 정부, 국민, 정책, 신뢰,...",1,국토교통부,국토교통부
1,"[재건축, 안전, 진단, 적용, 철회, 안전, 진단, 강화, 규정, 안전, 진단, ...",1,국토교통부,국토교통부
2,"[재건축, 안전, 진단, 적용, 철회, 안전, 진단, 강화, 규정, 안전, 진단, ...",1,국토교통부,국토교통부
3,"[남양주, 왕숙, 도시, 성공, 여부, 교통, 대책, 효율, 교통, 체증, 분산, ...",2,한국토지주택공사,한국토지주택공사
4,"[장지, 입체, 사업, 관련, 협의, 체, 구성, 사업, 피해, 인접, 단지, 대표...",35,서울특별시,서울특별시
...,...,...,...,...
99995,"[민원, 유입, 경로, 모바일, 사고, 발생, 지역, 서울특별시, 구로구, 구로동,...",177,서울특별시 구로구,서울특별시 구로구
99996,"[민원, 유입, 경로, 모바일, 사고, 발생, 지역, 경상남도, 양산시, 중부동, ...",3,경상남도 양산시,경상남도 양산시
99997,"[블럭, 주, 입구, 하나, 추가, 하나, 수, 블럭, 교통, 혼잡, 야기, 것, 문]",2,한국토지주택공사,한국토지주택공사
99998,"[민원, 유입, 경로, 모바일, 사고, 발생, 지역, 경기도, 화성시, 신동, 동탄...",22,경기도 화성시,경기도 화성시


왜 안되는걸까???? 학습에 사용한 데이터로 해보자

### 압축했으니까 밑부분은 안해도 됨